In [7]:
import os
import numpy as np
import pandas as pd
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [8]:
def get_embedding(text, model="text-embedding-3-small"):
   # Generates embedding for a text
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
 
def get_embeddings(
    list_of_text: list[str], model="text-embedding-3-small", **kwargs) -> list[list[float]]:
    assert len(list_of_text) <= 2048, "The batch size should not be larger than 2048."

    # replace newlines, which can negatively affect performance.
    list_of_text = [text.replace("\n", " ") for text in list_of_text]

    data = client.embeddings.create(input=list_of_text, model=model, **kwargs).data
    return [d.embedding for d in data]

In [37]:
#df = pd.read_csv("words.csv")
#df

In [38]:
# Returns embeddings for the words in 'words.csv' and stores them in 'embedding'. Saves this as a new csv file
#df['embedding'] = df['text'].apply(get_embedding)
#df.to_scv('word_embeddings.csv')

In [39]:
#df = pd.read_csv('word_embeddings.csv')
#df['embedding'] = df['embedding'].apply(eval).apply(np.array)
#df

In [40]:
# Load wanted data. Here is a speech
df = pd.read_csv("fed-speech.csv")
df.head()

,text
0,Good afternoon
1,My colleagues and I are strongly committed to ...
2,We have both the tools that we need and the re...
3,Price stability is the responsibility of the F...
4,"Without price stability, the economy does not ..."


In [41]:
# Get the embeddings of the speech. Save to csv to reduce embedding function calls
#df['embeddings'] = df['text'].apply(get_embedding)
#df.to_csv('fed-speech-embeddings')

In [89]:
# Load the csv file
df = pd.read_csv('fed-speech-embeddings')
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array) # What does this do??
df.head()

,Unnamed: 0,text,embeddings
0,0,Good afternoon,"[0.03404167294502258, -0.030998127534985542, -..."
1,1,My colleagues and I are strongly committed to ...,"[0.011237340047955513, 0.014179651625454426, 0..."
2,2,We have both the tools that we need and the re...,"[0.010302674025297165, 0.01715744100511074, 0...."
3,3,Price stability is the responsibility of the F...,"[-0.013447479344904423, -0.006005288101732731,..."
4,4,"Without price stability, the economy does not ...","[-0.01848752796649933, 0.005304287187755108, 0..."


In [94]:
# Search for something in the speech and create an embedding of the search sentence
search_sentence = "august"
search_embedded = get_embedding(search_sentence)

In [95]:
# Compute similarities between the speech and the search
df['similarities'] = df['embeddings'].apply(lambda x : cosine_similarity(x,search_embedded))
df.head()

,Unnamed: 0,text,embeddings,similarities
0,6,"Today, the FOMC raised our policy interest rat...","[-0.005314371082931757, -0.00754205510020256, ...",0.109952
1,33,"At today's meeting, the committee raised the t...","[0.007675030268728733, 0.017215684056282043, 0...",0.067248
2,52,We at the Fed will do everything we can to ach...,"[-0.0073754191398620605, -0.006112131755799055...",0.115736
3,34,"With today's action, we've raised interest rat...","[-0.007713939994573593, 0.012854378670454025, ...",0.143139
4,35,We anticipate that ongoing increases in the ta...,"[0.020980069413781166, 0.02223769761621952, 0....",0.123641


In [96]:
# Sort by most similar
df = df.sort_values("similarities", ascending=False)
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,text,embeddings,similarities
0,19,Job gains have been robust with employment ris...,"[-0.0012229527346789837, 0.062450945377349854,...",0.346641
1,0,Good afternoon,"[0.03404167294502258, -0.030998127534985542, -...",0.201786
2,13,Recent indicators point to modest growth of sp...,"[0.04072889685630798, 0.04065309837460518, 0.0...",0.180716
3,23,"Over the 12 months ending in September, total ...","[-0.028180072084069252, 0.04317083954811096, 0...",0.177496
4,24,The recent inflation data again have come in h...,"[0.016138914972543716, -0.0019731884822249413,...",0.152362


In [97]:
# Print top three results
for i in [0,1,2]:
    text = df['text'][i]
    print(f'Suggestion {i}: {text}')
    print('\n')

Suggestion 0: Job gains have been robust with employment rising by an average of 289,000 jobs per month over August and September


Suggestion 1: Good afternoon


Suggestion 2: Recent indicators point to modest growth of spending and production this quarter




In [20]:
def get_most_similar(text, df, n=3):
    search_embedded = get_embedding(text)
    df['similarities'] = df['embeddings'].apply(lambda x : cosine_similarity(x,search_embedded))
    df = df.sort_values("similarities", ascending=False)
    df = df.reset_index(drop=True)
    for i in range(n):
        print(f'Suggestion {i}: {df["text"][i]}')


def get_most_similar_unique(text, df, n=1):
    #  Split the input text into unique words
    words = set(text.split())
    # Iterate over each word and get the most similar embeddings
    for word in words:
        get_most_similar(word, df, n)
        
def load_data(file):
    df = pd.read_csv(file)
    df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)
    return df

In [21]:
search = "PCE fomc august prices"
df = load_data('fed-speech-embeddings')

In [18]:
get_most_similar(search,df)

Suggestion 0: Over the 12 months ending in September, total PCE prices rose at 6.2%, excluding the volatile food and energy categories, core PCE prices rose at 5.1%
Suggestion 1: Today, the FOMC raised our policy interest rate by 75 basis points and we continue to anticipate that ongoing increases will be appropriate
Suggestion 2: We at the Fed will do everything we can to achieve our maximum employment and price stability goals


In [19]:
get_most_similar_unique(search,df)

Suggestion 0: Today, the FOMC raised our policy interest rate by 75 basis points and we continue to anticipate that ongoing increases will be appropriate
Suggestion 0: Over the 12 months ending in September, total PCE prices rose at 6.2%, excluding the volatile food and energy categories, core PCE prices rose at 5.1%
Suggestion 0: Job gains have been robust with employment rising by an average of 289,000 jobs per month over August and September
Suggestion 0: Price pressures remained evident across a broad range of goods and services
